This notebook was automatically generated from your MAST-ML run so you can recreate the
plots. Some things are a bit different from the usual way of creating plots - we are
using the [object oriented
interface](https://matplotlib.org/tutorials/introductory/lifecycle.html) instead of
pyplot to create the `fig` and `ax` instances.


In [ ]:
"""
This module contains a collection of functions which make plots (saved as png files) using matplotlib, generated from
some model fits and cross-validation evaluation within a MAST-ML run.

This module also contains a method to create python notebooks containing plotted data and the relevant source code from
this module, to enable the user to make their own modifications to the created plots in a straightforward way (useful for
tweaking plots for a presentation or publication).
"""

import math
import os
import pandas as pd
import itertools
import warnings
import logging
from collections import Iterable
from os.path import join
from collections import OrderedDict

# Ignore the harmless warning about the gelsd driver on mac.
warnings.filterwarnings(action="ignore", module="scipy",
                        message="^internal gelsd")
# Ignore matplotlib deprecation warning (set as all warnings for now)
warnings.filterwarnings(action="ignore")

import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve

import matplotlib
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure, figaspect
from matplotlib.animation import FuncAnimation
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes

# Needed imports for ipynb_maker
from mastml.utils import nice_range
from mastml.metrics import nice_names

import inspect
import textwrap
from pandas import DataFrame, Series

import nbformat

from functools import wraps

matplotlib.rc('font', size=18, family='sans-serif') # set all font to bigger
matplotlib.rc('figure', autolayout=True) # turn on autolayout

# adding dpi as a constant global so it can be changed later
DPI = 250

log = logging.getLogger() # only used inside ipynb_maker I guess



In [ ]:
def stat_to_string(name, value):
    """
    Method that converts a metric object into a string for displaying on a plot

    Args:

        name: (str), long name of a stat metric or quantity

        value: (float), value of the metric or quantity

    Return:

        (str), a string of the metric name, adjusted to look nicer for inclusion on a plot

    """

    " Stringifies the name value pair for display within a plot "
    if name in nice_names:
        name = nice_names[name]
    else:
        name = name.replace('_', ' ')

    # has a name only
    if not value:
        return name
    # has a mean and std
    if isinstance(value, tuple):
        mean, std = value
        return f'{name}:' + '\n\t' + f'{mean:.3f}' + r'$\pm$' + f'{std:.3f}'
    # has a name and value only
    if isinstance(value, int) or (isinstance(value, float) and value%1 == 0):
        return f'{name}: {int(value)}'
    if isinstance(value, float):
        return f'{name}: {value:.3f}'
    return f'{name}: {value}' # probably a string


def plot_stats(fig, stats, x_align=0.65, y_align=0.90, font_dict=dict(), fontsize=14):
    """
    Method that prints stats onto the plot. Goes off screen if they are too long or too many in number.

    Args:

        fig: (matplotlib figure object), a matplotlib figure object

        stats: (dict), dict of statistics to be included with a plot

        x_align: (float), float denoting x position of where to align display of stats on a plot

        y_align: (float), float denoting y position of where to align display of stats on a plot

        font_dict: (dict), dict of matplotlib font options to alter display of stats on plot

        fontsize: (int), the fontsize of stats to display on plot

    Returns:

        None

    """

    stat_str = '\n'.join(stat_to_string(name, value)
                           for name,value in stats.items())

    fig.text(x_align, y_align, stat_str,
             verticalalignment='top', wrap=True, fontdict=font_dict, fontproperties=FontProperties(size=fontsize))


def make_fig_ax(aspect_ratio=0.5, x_align=0.65, left=0.10):
    """
    Method to make matplotlib figure and axes objects. Using Object Oriented interface from https://matplotlib.org/gallery/api/agg_oo_sgskip.html

    Args:

        aspect_ratio: (float), aspect ratio for figure and axes creation

        x_align: (float), x position to draw edge of figure. Needed so can display stats alongside plot

        left: (float), the leftmost position to draw edge of figure

    Returns:

        fig: (matplotlib fig object), a matplotlib figure object with the specified aspect ratio

        ax: (matplotlib ax object), a matplotlib axes object with the specified aspect ratio

    """
    # Set image aspect ratio:
    w, h = figaspect(aspect_ratio)
    fig = Figure(figsize=(w,h))
    FigureCanvas(fig)

    # Set custom positioning, see this guide for more details:
    # https://python4astronomers.github.io/plotting/advanced.html
    #left   = 0.10
    bottom = 0.15
    right  = 0.01
    top    = 0.05
    width = x_align - left - right
    height = 1 - bottom - top
    ax = fig.add_axes((left, bottom, width, height), frameon=True)
    fig.set_tight_layout(False)
    
    return fig, ax


In [ ]:
def plot_residuals_histogram(y_true, y_pred, savepath,
                             stats, title='residuals histogram', label='residuals'):
    """
    Method to calculate and plot the histogram of residuals from regression model

    Args:

        y_true: (numpy array), array of true y data values

        y_pred: (numpy array), array of predicted y data values

        savepath: (str), path to save the plotted precision-recall curve

        stats: (dict), dict of training or testing statistics for a particular run

        title: (str), title of residuals histogram

        label: (str), label used for axis labeling

    Returns:

        None

    """

    # make fig and ax, use x_align when placing text so things don't overlap
    x_align = 0.64
    fig, ax = make_fig_ax(x_align=x_align)

    #ax.set_title(title)
    # do the actual plotting
    residuals = y_true - y_pred

    #Output residuals data and stats to spreadsheet
    path = os.path.dirname(savepath)
    pd.DataFrame(residuals).describe().to_csv(os.path.join(path,'residual_statistics.csv'))
    pd.DataFrame(residuals).to_csv(path+'/'+'residuals.csv')

    #Get num_bins using smarter method
    num_bins = get_histogram_bins(y_df=residuals)
    ax.hist(residuals, bins=num_bins, color='b', edgecolor='k')

    # normal text stuff
    ax.set_xlabel('Value of '+label, fontsize=16)
    ax.set_ylabel('Number of occurences', fontsize=16)

    # make y axis ints, because it is discrete
    #ax.yaxis.set_major_locator(MaxNLocator(integer=True))

    plot_stats(fig, stats, x_align=x_align, y_align=0.90)
    plot_stats(fig, pd.DataFrame(residuals).describe().to_dict()[0], x_align=x_align, y_align=0.60)

    fig.savefig(savepath, dpi=DPI, bbox_inches='tight')


In [ ]:
from numpy import array
from collections import OrderedDict
from io import StringIO
y_true = array([177.28302038, 249.54944341, 210.37520634, 200.26888144,
       239.2147742 , 296.50121028, 276.21736875, 193.72169364,
       193.03789201, 396.88333541,   0.        ,  64.76678569,
       146.59493555, 143.14427   , 129.67655739, 185.18233461,
       176.19335258,  67.39021938, 113.27094167, 149.58971125,
       117.55701155,  85.32116056,  97.75371527,  88.06968527,
        89.11498671, 233.37177602, 181.75592179, 171.80996061,
       181.77618797, 172.42020306, 178.66988561, 125.7729607 ,
       369.78013418, 306.66689715, 335.55187932, 124.02294655,
       119.65758775, 116.8908945 , 127.740217  ,  80.94033473,
       187.42767666, 158.36740952, 291.95555798,  45.7531836 ,
        24.89260867,  87.31329665, 120.21481561, 270.00791293,
       106.61645203, 156.22972982,  71.90056154,  86.43036922,
        84.20250562, 113.13530223, 201.81756512, 198.06185912,
        73.49782973, 163.29786494,  69.43614177,  92.45610791,
        34.7342257 ,  29.45910878, 158.21863233,  30.40327127,
       114.52532705,  36.52821981,  50.0993    ,   0.        ,
         0.        ,   0.        ,   0.        , 307.83053925,
       140.13445715,  29.91808225,  51.86444618, 112.0035505 ,
       218.9463975 , 145.19783167, 202.6896798 , 219.40213351,
       117.65052986,   0.        ,  34.47065573,  60.12311255,
         0.        ,  29.94908075,  55.71532531,  25.83894891,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   1.92094295,   0.        ,
         0.        ,  27.03043917,  12.63281495,  11.84797082,
        16.57505557,   1.747132  ,  56.75173357,  44.85051669,
        48.99432914,  43.99569352,  39.22644569, 162.30858215,
       133.55965834, 140.19831868, 150.50818565,  86.40676351,
       115.20029363,   0.        ,  30.46897905,   0.        ,
        29.653255  ,   0.        ,   9.52352009,   0.        ,
         0.        ,  13.80460838,  52.62963569,  34.51982144,
        62.11081062, 118.54297288,  28.81237225,  24.13350925,
        95.52052281,   0.        ,  47.61927987,  35.46209575,
        26.16951837,  48.57118688,  73.3952221 , 117.71448387,
        36.257048  , 157.59559963, 100.85978128,  85.19420825,
        75.88549386, 187.39104625,   0.        , 137.12605894,
       170.3886173 ,   0.        ,   0.        ,  69.47351553,
       284.89818957,  44.54144834,  99.7625282 , 185.52454231,
        52.02867683, 202.66455344, 140.53855037, 134.5064424 ,
       139.34508133,  49.69176246,  73.13416251,  57.38221378,
        47.801376  ,  38.14651548,  36.16956355,   0.        ,
        26.23033875,  32.97595651,  22.41961241,   0.        ,
         0.        ,  55.47285962,   0.        ,   0.        ,
        44.84888737,  48.37559255,  58.54651811,  17.0719435 ,
         0.        ,  47.54553553,  91.38113959,  74.56373534,
       135.99188516,  56.54071259,  58.73687405,  58.3323071 ,
        76.83212346,  50.298441  ,  37.03306019,  42.81588044,
        34.01111394,  31.35144814,   0.        ,   0.        ,
        65.03154095,  18.9559087 , 134.6716424 ,  91.34499746,
        60.72512971,   0.        ,  65.98496318,  94.30190586,
        36.18162447,  26.50926775,  26.74964638,  42.90406645,
        10.35277113,  28.72421875,  33.67375834,  21.819592  ,
        66.76133312,  95.22684338,  42.34404359,   0.        ,
        14.00114963,   0.        ,   5.43550612,  35.72806622,
       178.13772044,  23.22818913,   7.05592531,   9.36490231,
       103.74396875, 133.03032487, 142.33395365, 157.24298149,
       245.69243888, 250.25203832, 260.12973722, 207.90480672,
       234.66065082, 275.65617815,  99.68134329, 187.87580019,
       252.76716229, 243.428371  , 224.62287057, 153.2465035 ,
       186.02653043, 114.50715188, 130.34718776, 114.4981617 ,
       122.77257944, 130.39035205, 189.88100424, 170.62748097,
       292.39066312, 100.86488318, 123.24662923, 122.43167022,
       205.09292614, 191.51236301, 227.87867171, 201.32142318,
       111.02632779, 129.31911366,  76.97840947, 197.12412955,
       195.67275424,  91.28475717,  88.62675986, 161.29335072,
       247.00509507,  96.92679621, 192.08874679, 198.5926738 ,
       141.92110132,  94.43350892, 234.968862  , 216.45461264,
       313.72288064, 160.23398776, 259.08638129, 354.73775188,
       353.34957447, 179.99933217, 317.205543  , 328.54172252,
       346.90203425, 164.00223776, 120.40352958, 141.45345548,
        68.40090273,  55.94984366,  89.54726266, 244.80034507,
       151.8983111 , 165.1097297 ,  64.5557199 , 158.20291287,
        77.46656255, 133.88336745, 132.39243296,  53.12824467,
       158.4134052 , 238.53505817, 324.94508429, 148.34054822,
       313.927543  , 228.16969945, 347.29590062, 857.23166531,
       142.41612109, 105.93237224,  95.77864058,  78.21525252,
        51.98836201,  77.08989732, 190.04640497,  94.39134167,
        96.47357254,  39.04655223,  17.43031391,  35.10096061,
        60.78290984, 113.32195047,  49.6708469 , 112.90930286,
       127.66464503,  68.87735374,  30.53286251, 133.6354298 ,
        58.54469733,  27.21757246, 187.84859075, 135.8400834 ,
       121.55357738, 211.33526584, 184.77929222, 163.40928559,
       115.40786595, 179.71035609, 137.40668645,  33.28049783,
        77.38602769,  83.56862417,  90.60897065, 174.06027747,
       113.09210788, 183.15120004, 397.23996543, 237.58337883,
       117.14051442, 138.58290334, 140.36478149, 144.11116592,
       244.71754232,  88.68589559, 263.54337143,  44.18158824,
        28.61719851,  46.91723712,   4.45796202,  38.63286983,
       165.73307859,  72.67526425,  61.69987974, 341.670362  ,
         0.        ,   0.        , 109.97295248, 161.96626836,
       179.36951362, 182.44993004, 110.11022338,  84.58123286,
         0.        ,  66.26244234,  82.91347525,   6.92239663,
         0.        ,  21.50586649,   4.75557533,  28.36967345,
         0.        ,  61.55780711,  28.48129225,   0.        ,
         0.        ,  84.72896518,  45.6348093 ,  27.12929863,
        14.17874738,  26.70381011,  33.75589221,  37.46802334,
        81.726855  ,  68.97106961, 166.79781986,  52.14150703,
         0.        ,  55.08861603,  45.03777853,  44.40936128,
        48.24004019,  38.17424353,  33.9806022 ,  43.94549002,
         0.        , 190.87087845,  82.62024503,  82.58263368,
        75.67939565, 157.61174657,  70.94302761,  36.22347961,
         0.        ,  90.16563994, 126.61616369,  28.07241153,
        30.74580969, 120.86584011,  29.23073416,  36.61104469,
        42.696629  ,  70.04323975,  17.00030381,  52.42546125,
        46.69821725,  33.46241925,  27.90730325, 121.7771504 ,
        38.754046  ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,  33.52619036,  23.809066  ,
         0.        ])
y_pred = array([0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       1.12550486e+002, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       5.94634793e-199, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 8.30752892e-199,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 4.68767987e+001, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 1.18032895e-316,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000])
savepath = '/home/nerve/Desktop/skunkworks/lane_schultz/hyper_parameters/test/DoNothing/DoNothing/KernelRidge_DoNothing_DoNothing_GridSearch/RepeatedKFold/split_2/test_residuals_histogram.png'
stats = OrderedDict([('R2', 0.0015697465373054609), ('root_mean_squared_error', 139.6196992842415), ('mean_absolute_error', 103.18265068806386), ('rmse_over_stdev', 1.4587431876436394)])
title = 'test_residuals_histogram'
label = 'Energy above convex hull (meV/atom)'

In [ ]:
import pandas as pd
from IPython.display import Image, display

plot_residuals_histogram(y_true, y_pred, savepath, stats, title, label)
display(Image(filename='test_residuals_histogram.png'))
